In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time, os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
os.getcwd()

'/Users/kunjian/Downloads/final-project-ascending'

In [3]:
# Load the links we want to scrap

data = pd.read_csv('data/greatschools/schools_data_all_states.csv')
urls = data.loc[:, ['universal-id', 'overview-url', 'state']].dropna()


url = 'https://www.greatschools.org/michigan/livonia/2340-Emerson-Middle-School#Students_with_Disabilities'
response = requests.get(url)


In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
soup


<!DOCTYPE html>

<html xmlns:fb="http://ogp.me/ns/fb#">
<head>
<title>Emerson Middle School - Livonia, Michigan - MI | GreatSchools</title>
<meta content="Emerson Middle School is a public school in Livonia, MI, offering Project Lead The Way curriculum. It is one of 6 middle schools in Livonia Public Schools School District." name="description"/>
<link href="https://www.greatschools.org/michigan/livonia/2340-Emerson-Middle-School/" rel="canonical"/>
<link href="https://www.greatschools.org/michigan/livonia/2340-Emerson-Middle-School/" hreflang="en" rel="alternate"/>
<link href="https://www.greatschools.org/michigan/livonia/2340-Emerson-Middle-School/?lang=es" hreflang="es" rel="alternate"/>
<meta content="Explore Emerson Middle School in Livonia, MI" property="og:title"/>
<meta content="We're an independent nonprofit that provides parents with in-depth school quality information." property="og:description"/>
<meta content="GreatSchools.org" property="og:site_name"/>
<meta content="htt

In [5]:
# Rating
blocks = soup.find_all('div', class_='gs-v2')
print(len(blocks))

for block in blocks:
    module_title = block.find('h2', class_='title-text')
    module_title = re.sub(r'\s+', ' ', module_title.text) if module_title else None
    if module_title:
        print(module_title)


18
Overview
School Rating
Rating Factors
Advanced Courses
Test Scores
Courses & Programs
Race/Ethnicity
Low-Income Students
Students With Disabilities
Student Demographics
Teachers & Staff
About the School
Neighborhood
Neighborhood


In [ ]:
import json

ratings_dict = {}

for block in blocks:
    module_title = block.find('h2', class_='title-text')
    module_title = re.sub(r'\s+', ' ', module_title.text) if module_title else None

    if module_title and 'School Improvement' in module_title:
        
        imp_dict = {}

        improvements = block.find('script', class_='js-react-on-rails-component')
        improvements = improvements.text if improvements else None
        # read the json string using json.loads and get the data
        improvements = json.loads(improvements)['data']

        for improvement in improvements:
            category = improvement.get('name', None)
            if category:
                score = improvement.get('school_value', 0) # for rubustness
                imp_dict[category] = score

        if imp_dict:
            ratings_dict['school_improvement'] = imp_dict

    if module_title and 'Student Demographics' in module_title:

        dem_dict = {}

        demographics = block.find('script', class_='js-react-on-rails-component')
        demographics = demographics.text if demographics else None
        # read the json string using json.loads and get the data
        demographics = json.loads(demographics)['data']

        for demographic in demographics.keys():
            if demographic == 'ethnicity_data':
                eth_dict = {}
                for eth in demographics[demographic]:
                    eth_name = eth.get('breakdown', None)
                    if eth_name:
                        eth_value = eth.get('value', 0) # for rubustness
                        eth_dict[eth_name] = eth_value
                if eth_dict:
                    dem_dict['ethnicity'] = eth_dict
            
            if demographic == 'gender_data':
                gender_dict = {}
                genders = demographics[demographic].get('value', None)
                if genders:
                    for gender in genders:
                        gender_name = gender.get('breakdown', None)
                        if gender_name:
                            gender_value = gender.get('school_value', 0) # for rubustness
                            gender_dict[gender_name] = gender_value
                if gender_dict:
                    dem_dict['gender'] = gender_dict

            if demographic == 'subgroups_data':
                subgroup_dict = {}
                subgroups = demographics[demographic].get('value', None)
                if subgroups:
                    for subgroup in subgroups:
                        subgroup_name = subgroup.get('breakdown', None)
                        if subgroup_name:
                            subgroup_value = subgroup.get('school_value', 0)
                            subgroup_dict[subgroup_name] = subgroup_value
                if subgroup_dict:
                    dem_dict['subgroups'] = subgroup_dict

        if dem_dict:
            ratings_dict['student_demographics'] = dem_dict
            
            
    if module_title and 'Teachers & Staff' in module_title:

        stf_dict = {}

        staffs = block.find('script', class_='js-react-on-rails-component')
        staffs = staffs.text if staffs else None
        # read the json string using json.loads and get the data
        staffs = json.loads(staffs)['data']
        
        for staff in staffs:
            category = staff.get('label', None)
            if category:
                score_school = staff.get('score', None)
                score_state = staff.get('state_average', None)

                # When the score is a dictionary, we need to get the value
                if type(score_school) == dict:
                    score_school = score_school.get('value', 0)
                if type(score_state) == dict:
                    score_state = score_state.get('value', 0)
                
                stf_dict[category] = {'school': score_school, 'state': score_state}
            
        if stf_dict:
            ratings_dict['teachers_staff'] = stf_dict
    
    if module_title and 'School Rating' in module_title:

        rating_dict = {}

        comparison = block.find('span', class_='strong').text
        score = block.find('div', class_='rs-gs-rating').text
        score = re.sub(r'\s+', ' ', score) if score else None

        rating_dict['score'] = score
        rating_dict['comparison'] = comparison

        if rating_dict:
            ratings_dict['school_rating'] = rating_dict

    if module_title and 'Rating Factors' in module_title:
        
        factors_dict = {}

        factors = block.find('script', class_='js-react-on-rails-component')
        factors = factors.text if factors else None
        # read the json string using json.loads and get the data
        factors = json.loads(factors).get('data', None)

        if factors:
            for factor in factors.get('subratings', None):
                factor_name = factor.get('title', None)
                if factor_name:
                    factor_score = factor.get('rating', 0)
                    factors_dict[factor_name] = factor_score

        if factors_dict:
            ratings_dict['rating_factors'] = factors_dict
        




NoneType

In [ ]:
# Make a request to the page and parse the html with BeautifulSoup

# Helper functions
def get_request(url):
    try:
        r = requests.get(url)
        if r.status_code == 200:
            return BeautifulSoup(r.text, 'html.parser')
        else:
            return None
    except:
        return None

